In [1]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
# re-size all the images to this
IMAGE_SIZE = [299, 299]

train_path = 'train'
valid_path = 'test'

In [3]:
myxception = Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [4]:
# don't train existing weights
for layer in myxception.layers:
    layer.trainable = False

In [5]:
  # useful for getting number of output classes
folders = glob('train/*')

In [6]:
print(len(folders))

16


In [7]:
# our layers - you can add more if you want
x = Flatten()(myxception.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=myxception.input, outputs=prediction)

In [9]:
# view the structure of the model
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255
                                 )

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (299, 299),
                                                 batch_size = 128,
                                                 class_mode = 'categorical')

Found 50024 images belonging to 16 classes.


In [13]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (299, 299),
                                            batch_size = 128,
                                            class_mode = 'categorical')

Found 11987 images belonging to 16 classes.


In [14]:
training_set.class_indices

{'atelectasis': 0,
 'cardiomegaly': 1,
 'consolidation': 2,
 'edema': 3,
 'effusion': 4,
 'emphysema': 5,
 'fibrosis': 6,
 'hernia': 7,
 'infiltration': 8,
 'lungopacity': 9,
 'mass': 10,
 'nodule': 11,
 'normal': 12,
 'pleural_thickening': 13,
 'pneumonia': 14,
 'pneumothorax': 15}

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=15,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/10
391/391 [==============================] - 3496s 9s/step - loss: 5.0684 - accuracy: 0.3528 - val_loss: 4.6309 - val_accuracy: 0.3244
Epoch 2/10
130/391 [========>.....................] - ETA: 24:27 - loss: 4.3902 - accuracy: 0.3887

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import load_model

model.save('multiclass_classification.h5')